In [135]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn import metrics

from sklearn.neighbors import KNeighborsClassifier #knn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

In [136]:
from sklearn.preprocessing import LabelEncoder

In [137]:
data= pd.read_csv("C:\\Users\\MASSIVE\\Price-Predictor-ML-\\final.csv")

In [138]:
print(data.columns)
data.isnull().sum()

Index(['id', 'title', 'price', 'specs_annee', 'specs_car_engine',
       'specs_couleur_auto', 'specs_papiers', 'specs_kilometrage',
       'specs_marque-voiture', 'specs_modele', 'specs_finition',
       'specs_energie', 'specs_boite', 'option0', 'option1', 'option2',
       'option3', 'option4', 'option5', 'option6', 'option7', 'option8',
       'option9', 'option10', 'option11', 'option12', 'option13'],
      dtype='object')


id                          0
title                       0
price                       0
specs_annee                 0
specs_car_engine         9798
specs_couleur_auto       2494
specs_papiers            4024
specs_kilometrage        4661
specs_marque-voiture        0
specs_modele                1
specs_finition          18647
specs_energie           92547
specs_boite             92547
option0                 17057
option1                 21209
option2                 23907
option3                 26781
option4                 29897
option5                 33568
option6                 37328
option7                 41555
option8                 46047
option9                 50978
option10                56118
option11                61725
option12                68319
option13                77359
dtype: int64

In [139]:
data.describe()

,id,price,specs_annee,specs_energie,specs_boite
count,9.254700e+04,9.254700e+04,92547.000000,0.0,0.0
mean,3.812892e+07,1.203689e+17,2012.852723,NaN,NaN
std,3.494813e+06,3.307373e+19,8.425591,NaN,NaN
min,5.065853e+06,0.000000e+00,1922.000000,NaN,NaN
25%,3.731984e+07,3.000000e+04,2010.000000,NaN,NaN
50%,3.892210e+07,1.330000e+06,2014.000000,NaN,NaN
75%,4.049664e+07,3.000000e+06,2019.000000,NaN,NaN
max,4.114435e+07,1.000000e+22,2024.000000,NaN,NaN


In [140]:
df_sans_redondance = data.drop_duplicates(subset=['title', 'price', 'specs_annee', 'specs_car_engine',
       'specs_couleur_auto', 'specs_papiers', 'specs_kilometrage',
       'specs_marque-voiture', 'specs_modele', 'specs_finition',
       'specs_energie', 'specs_boite', 'option0', 'option1', 'option2',
       'option3', 'option4', 'option5', 'option6', 'option7', 'option8',
       'option9', 'option10', 'option11', 'option12', 'option13'])

In [141]:
df_sans_redondance.describe()

,id,price,specs_annee,specs_energie,specs_boite
count,8.165100e+04,8.165100e+04,81651.000000,0.0,0.0
mean,3.801315e+07,1.364176e+17,2012.781178,NaN,NaN
std,3.610755e+06,3.521142e+19,8.432619,NaN,NaN
min,5.065853e+06,0.000000e+00,1922.000000,NaN,NaN
25%,3.725727e+07,3.000000e+04,2009.000000,NaN,NaN
50%,3.878533e+07,1.310000e+06,2014.000000,NaN,NaN
75%,4.046485e+07,3.000000e+06,2019.000000,NaN,NaN
max,4.114435e+07,1.000000e+22,2024.000000,NaN,NaN


In [142]:
copy=df_sans_redondance.copy()

In [143]:
# Create a new column named 'new_options' and initialize it with 0
copy['options'] = 0

# Iterate over each row
for index, row in copy.iterrows():
    options_count = 0
    # Count options from option0 to option13
    for i in range(14):
        option_col = f'option{i}'
        if not pd.isnull(row[option_col]):
            options_count += 1
    # Store the count of options in the 'new_options' column
    copy.at[index, 'options'] = options_count

In [144]:
copy.columns

Index(['id', 'title', 'price', 'specs_annee', 'specs_car_engine',
       'specs_couleur_auto', 'specs_papiers', 'specs_kilometrage',
       'specs_marque-voiture', 'specs_modele', 'specs_finition',
       'specs_energie', 'specs_boite', 'option0', 'option1', 'option2',
       'option3', 'option4', 'option5', 'option6', 'option7', 'option8',
       'option9', 'option10', 'option11', 'option12', 'option13', 'options'],
      dtype='object')

In [145]:
copy['options'].value_counts

<bound method IndexOpsMixin.value_counts of 0         0
1         1
2         7
3        11
4         0
         ..
92542    14
92543    14
92544     0
92545     7
92546    10
Name: options, Length: 81651, dtype: int64>

In [146]:
copy.drop(['option0', 'option1', 'option2',
       'option3', 'option4', 'option5', 'option6', 'option7', 'option8',
       'option9', 'option10', 'option11', 'option12', 'option13'], axis=1,inplace=True)

In [147]:
copy.columns

Index(['id', 'title', 'price', 'specs_annee', 'specs_car_engine',
       'specs_couleur_auto', 'specs_papiers', 'specs_kilometrage',
       'specs_marque-voiture', 'specs_modele', 'specs_finition',
       'specs_energie', 'specs_boite', 'options'],
      dtype='object')

In [148]:
print(copy[['id', 'title', 'price', 'specs_annee', 'specs_car_engine',
       'specs_couleur_auto', 'specs_papiers', 'specs_kilometrage',
       'specs_marque-voiture', 'specs_modele', 'specs_finition',
       'specs_energie', 'specs_boite']].dtypes)

id                        int64
title                    object
price                   float64
specs_annee               int64
specs_car_engine         object
specs_couleur_auto       object
specs_papiers            object
specs_kilometrage        object
specs_marque-voiture     object
specs_modele             object
specs_finition           object
specs_energie           float64
specs_boite             float64
dtype: object


In [149]:
copy['specs_kilometrage'] = pd.to_numeric(copy['specs_kilometrage'].str.replace('km', ''), errors='coerce')

In [150]:
copy['specs_annee'] = pd.to_numeric(copy['specs_annee'],errors='coerce')
copy['price'] = pd.to_numeric(copy['price'],errors='coerce')

In [151]:
copy['specs_boite'].value_counts()

Series([], Name: count, dtype: int64)

In [152]:
# Check for null values in the 'specs_energie' column
null_count = copy['specs_boite'].isnull().sum()
print("Number of null values:", null_count)

# Display unique values in the 'specs_energie' column
unique_values = copy['specs_boite'].unique()
print("Unique values:", unique_values)


Number of null values: 81651
Unique values: [nan]


In [153]:
copy['specs_papiers'] = copy['specs_papiers'].replace({
    'Carte grise / safia': 1,
    'Licence / Délai': 2,
    'Carte jaune': 3,
})

# Display the resulting DataFrame
print(copy['specs_papiers'].value_counts())

specs_papiers
1.0    73676
2.0     2875
3.0     1527
Name: count, dtype: int64


In [154]:
print(copy[['id', 'title', 'price', 'specs_annee', 'specs_car_engine',
       'specs_couleur_auto', 'specs_papiers', 'specs_kilometrage',
       'specs_marque-voiture', 'specs_modele', 'specs_finition',
       'specs_energie', 'specs_boite', 'options']].dtypes)

id                        int64
title                    object
price                   float64
specs_annee               int64
specs_car_engine         object
specs_couleur_auto       object
specs_papiers           float64
specs_kilometrage       float64
specs_marque-voiture     object
specs_modele             object
specs_finition           object
specs_energie           float64
specs_boite             float64
options                   int64
dtype: object


In [155]:
label_encoder = LabelEncoder()
copy['specs_couleur_auto'] = label_encoder.fit_transform(copy['specs_couleur_auto'])

In [156]:
copy['specs_marque-voiture'].value_counts()

specs_marque-voiture
Renault                  14962
Peugeot                  10856
Volkswagen                9256
Hyundai                   4875
Seat                      4394
                         ...  
Renault laguna               1
Dacia stpway sandero         1
Fiat une                     1
Renu kongo                   1
Ds7                          1
Name: count, Length: 634, dtype: int64

In [157]:
# Mapping dictionary to standardize brand names
brand_mapping = {
    'Renault laguna': 'Renault','رنوا': 'Renault','R4': 'Renault','Renu kongo':'Renault',
    'Dacia stpway sandero': 'Dacia','Dacia stpway sandero':'Dacia','Dacia stpway sandero':'Dacia',
    'Fiat une': 'Fiat','FIAT TIPO': 'Fiat',
    'Renu kongo': 'Renault','كليو': 'Renault',
    'Ds7': 'DS Automobiles',
    'Picanto' : 'Kia','Kia  sportage': 'Kia',
    'Ikco': 'Other',  # Correcting 'Ikco' to 'Other' (assuming it's not a valid brand)
    'chevroli': 'Chevrolet',  # Correcting 'chevroli' to 'Chevrolet'
    'Cheri': 'Chery','chiri': 'Chery',  # Correcting 'Cheri' to 'Chery'
    'DS Automobiles': 'DS' ,
    'ايبيزا 2015 2 بورت': 'Seat',
    'bmw e36':'BMW',
    'Picasso':'Citroen'
}

# Replace inconsistent brand names with the correct ones
copy['specs_marque-voiture'].replace(brand_mapping, inplace=True)

In [158]:
# Check the updated value counts
print(copy['specs_marque-voiture'].value_counts())

specs_marque-voiture
Renault           14966
Peugeot           10856
Volkswagen         9256
Hyundai            4875
Seat               4395
                  ...  
clio 3                1
Picanto               1
Other                 1
FISPA                 1
DS Automobiles        1
Name: count, Length: 620, dtype: int64


In [159]:
unique_brands = copy['specs_marque-voiture'].unique()

# Print the list of unique brands
print(unique_brands)

['Dacia' 'Seat' 'Kia' 'Nissan' 'Volkswagen' 'Peugeot' 'Fiat' 'Renault'
 'Chery' 'Great Wall' 'Citroen' 'Hyundai' 'Suzuki' 'Alfa Romeo'
 'Chevrolet' 'Mercedes' 'BMW' 'DS' 'Toyota' 'Huanghai' 'Vms' 'Skoda'
 'Daewoo' 'Balingo' 'Haima' 'Zotye' 'Audi' 'Mini Cooper' 'Gonow'
 'Land Rover' 'DFSK' 'Ibiza' 'SsangYong' 'QQ' 'JAC' 'Ford' 'Lada' 'Chana'
 'Opel' 'Baic' 'Honda' 'MG' 'CERTA' 'Faw' 'Chrysler' 'DFM'
 'Fiat Professional' 'Daihatsu' 'Hafei motors' 'Sonacom' 'Isuzu' 'Lagona'
 'Pougou' 'Desactivation adblue sur kia sportage et hyundai tucson'
 'رونو 18براك' 'Geely' 'Mazda' 'Kango' 'Pertner'
 'Yamaha svho super charge' 'Lifan' 'Maruti' 'Ford fiesta' 'Tata'
 'Gonow dfsk dfm sokon chana hafei faw' 'Rover' 'Mitsubishi' 'Changan'
 'clio' 'Porsche' 'رونو   18' 'Aston Martin' 'Ssang yong' 'Tesla' 'Atos'
 'Brilliance' 'رونو' 'Maserati' 'Xsara' 'Smbole' 'Jaguar' 'بيجو' 'Sym'
 'terios' 'Dfsk dfm' 'فورد' 'شفرولي افيو' 'Golf' 'Partner' 'toureg' 'Cuxi'
 'Renault Kangoo' 'Victoria' 'Cadillac' 'Doblo' 'Cl

In [160]:
copy['Engine Capacity'] = copy['specs_car_engine'].str[:3]

In [161]:
copy['Engine Capacity'] = pd.to_numeric(copy['Engine Capacity'], errors='coerce')
copy.loc[copy['Engine Capacity'] > 26, 'Engine Capacity'] = pd.NA

# Calculate the mean of the "Engine Capacity" column, excluding NaN values
mean_value = copy['Engine Capacity'].mean(skipna=True)

# Fill NaN values in the "Engine Capacity" column with the calculated mean
copy['Engine Capacity'].fillna(mean_value, inplace=True)

In [162]:
print(copy['Engine Capacity'].value_counts())

Engine Capacity
1.61063    16608
1.60000    14712
2.00000    10372
1.50000     8660
1.20000     7530
           ...  
5.10000        1
0.70000        1
3.90000        1
6.20000        1
0.11000        1
Name: count, Length: 70, dtype: int64


In [163]:
copy.columns

Index(['id', 'title', 'price', 'specs_annee', 'specs_car_engine',
       'specs_couleur_auto', 'specs_papiers', 'specs_kilometrage',
       'specs_marque-voiture', 'specs_modele', 'specs_finition',
       'specs_energie', 'specs_boite', 'options', 'Engine Capacity'],
      dtype='object')

In [164]:
print(copy[['id', 'title', 'price', 'specs_annee', 'specs_car_engine',
       'specs_couleur_auto', 'specs_papiers', 'specs_kilometrage',
       'specs_marque-voiture', 'specs_modele', 'specs_finition',
       'specs_energie', 'specs_boite', 'options','Engine Capacity']].dtypes)

id                        int64
title                    object
price                   float64
specs_annee               int64
specs_car_engine         object
specs_couleur_auto        int32
specs_papiers           float64
specs_kilometrage       float64
specs_marque-voiture     object
specs_modele             object
specs_finition           object
specs_energie           float64
specs_boite             float64
options                   int64
Engine Capacity         float64
dtype: object


In [165]:
copy.describe().T

,count,mean,std,min,25%,50%,75%,max
id,81651.0,3.801315e+07,3.610755e+06,5065853.0,37257272.0,38785328.0,4.046485e+07,4.114435e+07
price,81651.0,1.364176e+17,3.521142e+19,0.0,30000.0,1310000.0,3.000000e+06,1.000000e+22
specs_annee,81651.0,2.012781e+03,8.432619e+00,1922.0,2009.0,2014.0,2.019000e+03,2.024000e+03
specs_couleur_auto,81651.0,1.083955e+01,7.801758e+00,0.0,3.0,10.0,1.500000e+01,3.100000e+01
specs_papiers,78078.0,1.075937e+00,3.305852e-01,1.0,1.0,1.0,1.000000e+00,3.000000e+00
specs_kilometrage,77616.0,1.289683e+28,3.589423e+30,-1340000.0,468.0,97148.0,2.300000e+05,1.000000e+33
specs_energie,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
specs_boite,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
options,81651.0,7.578144e+00,5.193321e+00,0.0,2.0,8.0,1.300000e+01,1.400000e+01
Engine Capacity,81651.0,1.610630e+00,6.185367e-01,0.0,1.4,1.6,1.610630e+00,2.500000e+01


In [174]:
column_values = copy['specs_car_engine'].dropna()  # Drop NaN values in the column

# Initialize an empty list to store the unique strings separated by a space
space_separated_strings = []

# Iterate over each value in the column
for value in column_values:
    # Split the value by space
    words = value.split()
    # Check if there are more than one word (separated by space) in the value
    if len(words) > 1:
        # Add each separated string to the list (excluding duplicates)
        for word in words:
            if word not in space_separated_strings:
                space_separated_strings.append(word)

# Print the list of unique str
print(space_separated_strings)

['1.5', 'DCI', '90ch', '1.6', 'MPi', '2.0', '150ch', '4WD', 'HDI', '92ch', '1.2', 'ess', '16V', '75ch', '72ch', 'TDI', '110ch', '80ch', '102ch', '143ch', '1.9', 'tdi', 'Dci', '85ch', 'dci', '0.8', '38ch', '1.4', '1.8', 'D', '65ch', '140ch', '103ch', '100ch', '1.1', '68ch', '84ch', 'VTI', '4X4', 'VCDI', 'BVA', '1.3', 'VVT', '94ch', '250', 'CDI', '204', '112ch', '55ch', 'DCi', '2.5', '171ch', '105ch', 'Ecomotive', '2.8', 'TD', '115ch', 'VVTI', '99ch', 'VM', '425', 'FAP', '115', 'CH', '80', 'ch', '47ch', 'CVVT', '87ch', 'ESS', '130', 'cv', 'hdi', '4X2', '91ch', '0.9', '34ch', '83ch', '71ch', '95ch', '60ch', '1.0', '70ch', '120ch', '109ch', '2.1', 'DT', 'CRDI', '177', '4x4', '200chv', '82ch', 'DSG', '6', '69ch', '16', 'dti', '63ch', '1,6', '53', '2x4', '127ch', 'S90', '_1.8', 'DDTI', '174ch', '170ch', '184ch', '220', '177ch', 'TDi', 'BlueEfficiency', 'simple', '130ch', 'GTI', '202ch', 'H', 'i', '92', 'Ch', 'V16', '2.2', 'SD4', '-', '190ch', 'Tdi', 'Bosch', 'Sans', 'tirbo', '85chv', '197ch'

In [177]:

provided_engine_names = ['HDI', 'TDI', 'VTEC', 'VVTi', 'TSI', 'TFSI', 'ECOBOOST', 'CRDi', 'MPI', 'GDI',
                        'V6', 'V8', 'V12', 'Boxer', 'Wankel', 'Rotary', 'Hybrid', 'Turbo', 'Supercharged',
                        'Diesel', 'Gasoline', 'Electric', 'Petrol', 'CNG', 'LPG', 'EcoDiesel', 'FlexFuel',
                        'Direct Injection', 'Common Rail', 'Variable Valve Timing', 'Variable Valve Lift',
                        'Variable Valve Duration', 'Overhead Cam', 'DOHC', 'SOHC', 'OHV', 'Pushrod',
                        'Hybrid Synergy Drive', 'SkyActiv', 'BlueHDi', 'BlueMotion', 'eTorque', 'EcoBoost',
                        'PureTech', 'EcoTec', 'i-VTEC', 'm-VTEC', 'Dual VVT-i', 'VVT-L', 'Variable Twin Turbo',
                        'ess','dci','DCi','Dci','tce','VVTI','tsi','VVTL','4WD','hDi']


# Assuming 'copy' is the name of your DataFrame and 'specs_car_engine' is the name of the column
# Replace 'specs_car_engine' with the actual name of your column
copy['Engine_Name_Match'] = copy['specs_car_engine'].apply(lambda x: next((engine for engine in provided_engine_names if isinstance(x, str) and engine.lower() in x.lower()), pd.NA))

# Display the DataFrame with the new column
print(copy)


             id                                        title      price  \
0      37481055                   Dacia Sandero 2019 Stepway  3350000.0   
1      37480999                Seat Ibiza 2018 High Facelift  1230000.0   
2      37480705                          Kia Picanto 2023 Ex  3950000.0   
3      37229272                         Nissan Qashqai 2009   1600000.0   
4      37479579  Volkswagen Transporteur 2 T2 1982 Brésilien   380000.0   
...         ...                                          ...        ...   
92542  41120407                   Volkswagen Golf 8 2022 GTD  7800000.0   
92543  41118613                       Hyundai Accent 2020 RB  3180000.0   
92544  41120392                         Fiat Doblo 2024 Life  3600000.0   
92545  41120380                            Peugeot 207 2012   1900000.0   
92546  41118598                Volkswagen Golf 2 1990 Golf 2   990000.0   

       specs_annee   specs_car_engine  specs_couleur_auto  specs_papiers  \
0             2019     

In [178]:
print(copy['Engine_Name_Match'].value_counts())

Engine_Name_Match
ess            18969
TDI             8483
HDI             7758
dci             7300
CRDi            2234
MPI             1850
TSI              818
tce              497
Turbo            452
VVTi             425
TFSI             340
Diesel           168
Hybrid           135
V6               112
DOHC              80
V8                71
PureTech          29
GDI               28
ECOBOOST          11
VTEC               9
4WD                7
Common Rail        5
eTorque            2
V12                2
EcoTec             2
Name: count, dtype: int64


In [168]:
copy.columns

Index(['id', 'title', 'price', 'specs_annee', 'specs_car_engine',
       'specs_couleur_auto', 'specs_papiers', 'specs_kilometrage',
       'specs_marque-voiture', 'specs_modele', 'specs_finition',
       'specs_energie', 'specs_boite', 'options', 'Engine Capacity',
       'Engine_Name_Match'],
      dtype='object')

In [169]:
print(copy[['id', 'title', 'price', 'specs_annee', 'specs_car_engine',
       'specs_couleur_auto', 'specs_papiers', 'specs_kilometrage',
       'specs_marque-voiture', 'specs_modele', 'specs_finition',
       'specs_energie', 'specs_boite', 'options', 'Engine Capacity',
       'Engine_Name_Match']].dtypes)

id                        int64
title                    object
price                   float64
specs_annee               int64
specs_car_engine         object
specs_couleur_auto        int32
specs_papiers           float64
specs_kilometrage       float64
specs_marque-voiture     object
specs_modele             object
specs_finition           object
specs_energie           float64
specs_boite             float64
options                   int64
Engine Capacity         float64
Engine_Name_Match        object
dtype: object
